## FINAL PROJECT


Imports

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re #regular expressions library, 
%matplotlib inline
import pandas as pd
import numpy as np
## plotting libraries
from matplotlib import pyplot as plt
import seaborn as sns
## Sklearn libraries
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline



In [ ]:
data = pd.read_csv("TABELA_GERAL_COL7_MAPBIOMAS_DESMAT_VEGSEC_UF.csv") #to predict what category the news belongs
data.head()

In [ ]:
data = data.drop(['Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50','Unnamed: 51'], axis = 1)
data.tail()

In [ ]:
data = data.drop(['level_0', 'level_1', 'level_2', 'level_3','level_4','color','feature_id','dr_class_id','class_id','1986','1987','1988','1989','1990','1991','1992','1993','1994','1995','1996','1997','1998'], axis = 1)
data = data.reset_index(drop=True)
data.tail()

In [ ]:
data = data[:40606]
data

In [ ]:
threshold = 0.65
data = data.dropna(thresh=len(data)* (1-threshold), axis=1)
data

In [ ]:
from skimpy import clean_columns
from nltk.metrics import edit_distance
   
def fill_nans_with_means(df):
    return df.fillna(df.mean()).round()

x = fill_nans_with_means(data)
x

In [ ]:
ge = x['city'].str.split(' - ', expand=True)
ge = ge.rename(columns={0: 'City', 2: 'Bioma'})
x['city'] = ge['City']
x = x.join(ge['Bioma'])

In [ ]:
x.rename(columns={'dr_class_name':'Deforestation'}, inplace=True)
x

In [ ]:
x = x[~x.isin(['Veg. Secundária', 'Supressão Veg. Primária','Recuperação para Veg. Secundária','Supressão Veg. Secundária','Outras transições']).any(axis=1)]
x

In [ ]:
x = x.groupby('city', as_index=False).sum()
x

### SECOND DATASET

In [ ]:
data1 = pd.read_csv("tabelao-setores-SEEG-Municipios-2_0-GWP-AR5-FINAL-SITE.csv", encoding = "ISO-8859-1") 
data1.tail()

In [ ]:
data1 = data1[~data1.isin(['Caatinga', 'Mata Atlântica','Pantanal','Pampa']).any(axis=1)]
data1

In [ ]:
data1.rename(columns={'setor':'sector', 'municipio':'city', 'bioma':'biome', 'tipo_emissao':'type_of_emission','atividade_economica':'Economic_Activity'}, inplace=True)
data1

In [ ]:
data1 = data1[~data1.isin(['Energia Elétrica', 'Aluminio e Outros Não Ferrosos da Metalurgia','Ferro e Aço']).any(axis=1)]
data1

In [ ]:
data1 = data1[~data1.isin(['Remoção', 'Bunker']).any(axis=1)]
data1

In [ ]:
data1 = data1[~data1.isin(['Cultivo de arroz','Emissões pela Queima de Combustíveis','Efluentes Liquidos']).any(axis=1)]
data1

In [ ]:
data1 = data1[~data1.isin(['Produção de Combustíveis', 'Outras Industrias','Cimento','Residencial','Transporte','Metalurgia','Saneamento Básico','Uso de HFCs']).any(axis=1)]
data1

In [ ]:
data1 = data1[~data1.isin(['Arroz', 'Feijão','Mandioca','Milho', 'Floresta primária -- Silvicultura', 'Floresta secundária -- Silvicultura','Vegetação não florestal secundária -- Silvicultura','Uso agropecuário -- Silvicultura' ]).any(axis=1)]
data1

In [ ]:
data1 = data1.drop(['nivel_2', 'nivel_3', 'nivel_4', 'nivel_5','nivel_6','uf', 'ibge_cod', 'produto','sector','type_of_emission','Econonomic_Activity'], axis = 1)
data1.tail()

In [ ]:
data1 = data1.groupby('city', as_index=False).sum()
data1

# FROM HERE

In [ ]:
import difflib

def substitute_names(list_a, list_b):
    for i, name_b in enumerate(list_b):
        for name_a in list_a:
            if difflib.SequenceMatcher(None, name_a, name_b).ratio() >= 0.7:
                list_b[i] = name_a
                break
    return list_b

substituted_list = substitute_names(data1['city'], x['city'])
substituted_list

In [ ]:
x.city